### 1. Setup OpenAI API Key
This cell imports required libraries and sets up your OpenAI API key.

In [ ]:
import openai
from openai import OpenAI
import json
import os
openai_api_key='your_openai_api_key_here'


### 2. Load Base RTL Designs
This function loads Verilog/VHDL files from a specified folder for modification.

In [2]:
def load_base_designs(directory):
    base_designs = []
    for file in os.listdir(directory):
        if file.endswith(".v"):
            with open(os.path.join(directory, file), "r") as f:
                base_designs.append((file, f.read()))
    return base_designs

### 3. Construct Trojan Injection Prompt
Here we build a prompt for the LLM describing where/how to insert the Trojan.

In [3]:
def construct_prompt(design, vulnerability, prompting_strategy):
    prompt = f"""
Analyze and modify the following Verilog code to insert a hardware Trojan:

<{design}>

Task: Insert a {vulnerability} Trojan using this strategy: {prompting_strategy}
Maintain original functionality while adding the Trojan. Ensure the design is fully synthesizable.

Synthesizable Design Guidelines
1. Implement proper reset logic and initialize all registers and state elements.
2. Employ counters for time/event-based triggers.
3. Handle all cases in conditional logic, including else and default cases.
4. Omit non-synthesizable constructs (initial blocks, $display).
5. Ensure all modifications are within synthesizable Verilog constructs.

Provide a response in the following format:

Code:
<Insert full modified Verilog code with comments highlighting Trojan insertions(must provide the complete code)>

Explanation:
<Two-line description of how the Trojan was inserted and its impact>

Trigger:
<Two-line description of the Trojan's trigger mechanism and activation conditions>

Payload:
<Two-line description of the Trojan's payload and its effects>

Taxonomy:
<Insertion phase: Design
Abstraction level: Register-transfer level
Activation mechanism: [Specify: Always-on, Triggered internally/externally, etc.]
Effects: {vulnerability}
Location: [Specify: Processor, Memory, I/O, Power Supply, Clock Grid]
Characteristics: [Specify: Distribution, Size, Type (Functional/Parametric)]>

Important Instruction: Ensure your response strictly adheres to this format.
CRITICAL INSTRUCTION: Provide only one instance of each section. Do not repeat or rephrase your response under any circumstances. Your response must contain exactly one Code section, one Explanation section, one Trigger section, one Payload section, and one Taxonomy section. Any repetition will result in an incorrect output.

"""
    return prompt

vulnerabilities = {
    "T1": "change functionality",
    "T2": "leak information",
    "T3": "denial of service",
    "T4": "performance degradation"
}

prompting_strategies = {
    "T1": "Subtly alter logic to produce incorrect results under specific conditions.",
    "T2": "Design a covert data transmission mechanism that activates upon detecting a specific signal pattern and ensure access to key internal signals.",
    "T3": "Introduce a condition that temporarily disables the module, triggered by a rare sequence of events.",
    "T4": "Implement a continously running shift register or accumulator to increase power consumption without affecting primary functionality upon detecting a specific event."
    # "T2": "Add a covert mechanism to transmit internal data, activated by a specific signal pattern.",
}


### 4. Send Prompt to OpenAI
This function sends the prompt to the LLM and retrieves the response.

In [4]:
#ChatGPT
from openai import OpenAI
client = OpenAI(api_key=openai_api_key)

def model_inference(prompt):
    model_name = "gpt-4.1" # choose: gpt-4 or gpt-4.1
    completion = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": "You are an expert skilled in hardware design and verification."},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content, model_name


### 5. Extract Code and Metadata
This function parses the LLM output and separates the code, explanations, and Trojan type.

In [5]:
def extract_code_and_metadata(response_text):
    sections = ["code:", "explanation:", "trigger:", "payload:", "taxonomy:"]
    results = {}

    response_lower = response_text.lower()
    for i, section in enumerate(sections):
        start = response_lower.find(section)
        if start == -1:
            print(f"Warning: '{section}' not found in the response.")
            results[section] = ""
            continue

        start += len(section)
        if i < len(sections) - 1:
            end = response_lower.find(sections[i+1], start)
            if end == -1:
                end = len(response_lower)
        else:
            end = len(response_lower)

        content = response_text[start:end].strip()

        # Clean up the content
        content = clean_content(content, section)

        results[section] = content

    return (results["code:"], results["explanation:"], results["trigger:"],
            results["payload:"], results["taxonomy:"])

def clean_content(content, section):
    # Remove triple backticks and language specifiers
    content = content.replace("```", "").strip()

    # Remove '##' markers
    content = content.replace("##", "").strip()

    # For code section, ensure it starts with `timescale, `include, or module
    if section == "code:":
        lines = content.split("\n")
        start_index = 0
        for i, line in enumerate(lines):
            if (line.strip().startswith("`timescale") or
                line.strip().startswith("`include") or
                line.strip().startswith("module")):
                start_index = i
                break
        content = "\n".join(lines[start_index:])

    # For non-code sections, take only the first paragraph to avoid repetition
    elif section != "taxonomy:":
        paragraphs = content.split("\n\n")
        content = paragraphs[0] if paragraphs else ""

    return content.strip()


In [6]:

import os

def create_directory(path):
    os.makedirs(path, exist_ok=True)

def save_vulnerable_design(design_name, verilog_code, base_directory, vulnerability_id, model_name, version_number):
    base_name = os.path.splitext(design_name)[0]
    directory = os.path.join(base_directory, model_name, base_name)
    create_directory(directory)
    filename = f"{base_name}_H{vulnerability_id}_{model_name}_A{version_number}.v"
    file_path = os.path.join(directory, filename)
    print(f"Saving vulnerable design to: {file_path}")

    # Remove any content after the last 'endmodule'
    last_endmodule_index = verilog_code.rfind('endmodule')
    if last_endmodule_index != -1:
        verilog_code = verilog_code[:last_endmodule_index] + 'endmodule'

    with open(file_path, "w") as f:
        f.write(verilog_code.strip())

def save_vulnerability_description(design_name, base_directory, vulnerability_id, explanation, trigger, payload, taxonomy, model_name, version_number):
    base_name = os.path.splitext(design_name)[0]
    directory = os.path.join(base_directory, model_name, base_name)
    create_directory(directory)
    description = f"""
Design: {design_name}
Vulnerability ID: {vulnerability_id}
Model: {model_name}
Attempts: {version_number}

Explanation:
{explanation}

Trigger:
{trigger}

Payload:
{payload}

Taxonomy:
{taxonomy}
"""
    filename = f"{base_name}_H{vulnerability_id}_{model_name}_A{version_number}_taxonomy.txt"
    file_path = os.path.join(directory, filename)
    print(f"Saving vulnerability description to: {file_path}")
    with open(file_path, "w") as f:
        f.write(description.strip())

In [7]:
!git clone -b crypto_core_aes https://github.com/fabriziotappero/ip-cores.git

Cloning into 'ip-cores'...
remote: Enumerating objects: 119893, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 119893 (delta 0), reused 0 (delta 0), pack-reused 119892 (from 2)
Receiving objects: 100% (119893/119893), 664.39 MiB | 21.32 MiB/s, done.
Resolving deltas: 100% (32133/32133), done.


In [8]:
!mkdir -p /content/aes

!cp /content/ip-cores/rtl/aes_128.v /content/aes/
!cp /content/ip-cores/rtl/round.v /content/aes/
!cp /content/ip-cores/rtl/table.v /content/aes/
load_base_designs("/content/aes")

[('round.v',
  '/*\n * Copyright 2012, Homer Hsing <homer.hsing@gmail.com>\n *\n * Licensed under the Apache License, Version 2.0 (the "License");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n * http://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an "AS IS" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\n/* one AES round for every two clock cycles */\nmodule one_round (clk, state_in, key, state_out);\n    input              clk;\n    input      [127:0] state_in, key;\n    output reg [127:0] state_out;\n    wire       [31:0]  s0,  s1,  s2,  s3,\n                       z0,  z1,  z2,  z3,\n                       p00, p01, p02, p03,\n                   

In [9]:
def main(version_number):
    base_designs_directory = "/content"  # Adjust path as needed
    vulnerable_designs_directory = "aes"  # Adjust path as needed

    print(f"Vulnerable designs directory: {os.path.abspath(vulnerable_designs_directory)}")
    print(f"Directory exists: {os.path.exists(vulnerable_designs_directory)}")

    base_designs = load_base_designs(base_designs_directory+"/"+vulnerable_designs_directory)

    for design_name, design in base_designs:
        for vulnerability_id, vulnerability in vulnerabilities.items():
            prompting_strategy = prompting_strategies[vulnerability_id]
            prompt = construct_prompt(design, vulnerability, prompting_strategy)

            response_text, model_name = model_inference(prompt)
            print(response_text)

            verilog_code, explanation, trigger, payload, taxonomy = extract_code_and_metadata(response_text)

            # print(f"Saving design: {design_name}")
            # print(f"Vulnerability ID: {vulnerability_id}")
            # print(f"Model name: {model_name}")

            try:
                save_vulnerable_design(design_name, verilog_code, vulnerable_designs_directory, vulnerability_id, model_name, version_number)
                save_vulnerability_description(design_name, vulnerable_designs_directory, vulnerability_id, explanation, trigger, payload, taxonomy, model_name, version_number)
            except Exception as e:
                print(f"Error saving files: {str(e)}")
                print(f"Design name: {design_name}")
                print(f"Vulnerability ID: {vulnerability_id}")
                print(f"Model name: {model_name}")
                print(f"Attempt number: {version_number}")
                raise  # Re-raise the exception to stop execution

    # print(f"Attempt {version_number}: Vulnerable designs and descriptions generated successfully!")

# Run the main function with the desired attempt number
for attempt in range(1, 2):  # Run 1 attempt for now
    main(version_number=attempt)

Vulnerable designs directory: /content/aes
Directory exists: True
Code:
```verilog
/*
 * Copyright 2012, Homer Hsing <homer.hsing@gmail.com>
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

/* one AES round for every two clock cycles with reset */
module one_round (
    input              clk,
    input              rst, // Added reset
    input      [127:0] state_in,
    input      [127:0] key,
    output reg [127:0] state_out
);

    wire [31:0]  s0,  s1,  s2,  s3,
                 z